In [1]:
# [+] SparkSession 임포트
import findspark
findspark.init()

from pyspark.sql import SparkSession

# [+] SparkSession 객체 생성 및 설정
spark = SparkSession.builder.master('local').appName('udf').getOrCreate()

In [2]:
# 샘플 데이터: 한신대 맛집 및 대표메뉴
hsu_restaurants = [
    ('진현가든', '삼치돌솥밥', '경기 오산시 양산로 340딩', 3.5),
    ('대광생막창', '돼지막창', '경기 화성시 한신대길 99', 4.0),
    ('찌개동아리', '제육전골', '경기 오산시 한신대133번길 4', 3.5),
    ('한판삼겹', '항정살', '경기 오산시 양산로410번길 8', 3.5),
    ('화락', '초밥', '경기 오산시 양산로 347 대성빌딩1층', 3.5),
    ('해우리', '해물라면', '경기 오산시 한신대길 135 1층', 4.5),
    ('행복한콩박사', '맑은순두부', '경기 오산시 양산로398번길 8-11', 3.5)
]


In [4]:
# [+] 스키마 정의
schema = ['restaurant_name', 'specialty', 'address', 'score']

In [5]:
# [+] 데이터프레임 생성
df = spark.createDataFrame(hsu_restaurants, schema)

In [6]:
# [+] 데이터프레임 출력
df.show()

+---------------+----------+------------------------------+-----+
|restaurant_name| specialty|                       address|score|
+---------------+----------+------------------------------+-----+
|       진현가든|삼치돌솥밥|      경기 오산시 양산로 340딩|  3.5|
|     대광생막창|  돼지막창|       경기 화성시 한신대길 99|  4.0|
|     찌개동아리|  제육전골|   경기 오산시 한신대133번길 4|  3.5|
|       한판삼겹|    항정살|   경기 오산시 양산로410번길 8|  3.5|
|           화락|      초밥|경기 오산시 양산로 347 대성...|  3.5|
|         해우리|  해물라면|  경기 오산시 한신대길 135 1층|  4.5|
|   행복한콩박사|맑은순두부|경기 오산시 양산로398번길 8-11|  3.5|
+---------------+----------+------------------------------+-----+



In [9]:
# [+] 데이터프레임 스키마 출력
df.printSchema()

root
 |-- restaurant_name: string (nullable = true)
 |-- specialty: string (nullable = true)
 |-- address: string (nullable = true)
 |-- score: double (nullable = true)



In [11]:
# [+] Temporary View 생성
df.createOrReplaceTempView('restaurants')

### User Defined Function 생성
1. translate(): Google Translation API를 이용하여 한글 식당 이름을 영문으로 번역하는 함수
2. scale_score(): 5점 만점 평점을 100점 스케일로 변환하는 함수

In [13]:
# Google translation 라이브러리 설치
!pip install googletrans==4.0.0-rc1

In [14]:
# Google translator 임포트
from googletrans import Translator

translator = Translator()

In [20]:
# 번역 테스트
result = translator.translate("찌개 동아리", src = 'ko', dest = 'en')
print(result)

Translated(src=ko, dest=en, text=Stew, pronunciation=None, extra_data="{'confiden...")


In [21]:
# [+] 번역 결과 출력
result.text

'Stew'

In [22]:
# [+] extra_data 출력
result.extra_data

{'confidence': None,
 'parts': [<googletrans.models.TranslatedPart at 0x20934382400>],
 'origin_pronunciation': 'jjigae dong-ali',
 'parsed': [['jjigae dong-ali',
   None,
   None,
   [[[0, [[[None, 6]], [True]]]], 6],
   [['찌개 동아리', None, None, 6]],
   None,
   ['찌개 동아리', 'ko', 'en', True]],
  [[[None,
     None,
     None,
     None,
     None,
     [['Stew',
       None,
       None,
       None,
       [['Stew', [5], []], ['Stew club', [11], []]]]]]],
   'en',
   1,
   'ko',
   ['찌개 동아리', 'ko', 'en', True]],
  'ko']}

In [23]:
# 영어 발음(pronunciation) 출력
result.extra_data.get('origin_pronunciation')

'jjigae dong-ali'

In [24]:
# UDF 1: 한글->영문 번역 함수
def translate(text):
    from googletrans import Translator
    translator = Translator()
    result = translator.translate(text, src = 'ko', dest = 'en')
    return result.extra_data.get('origin_pronunciation')

In [25]:
# UDF 등록
spark.udf.register('translate', translate)

<function __main__.translate(text)>

In [27]:
# SQL문 처리
spark.sql("SELECT restaurant_name, translate(restaurant_name) AS restaurant_name_en \
            FROM restaurants").show()

+---------------+--------------------+
|restaurant_name|  restaurant_name_en|
+---------------+--------------------+
|       진현가든|      jinhyeongadeun|
|     대광생막창|daegwangsaengmagc...|
|     찌개동아리|      jjigaedong-ali|
|       한판삼겹|      hanpansamgyeob|
|           화락|              hwalag|
|         해우리|              haeuli|
|   행복한콩박사|haengboghankongbagsa|
+---------------+--------------------+



In [29]:
# Annotation 방식으로 UDF 등록하기
from pyspark.sql.functions import udf

In [52]:
# UDF 2: score -> 100점 스케일 변환 함수
@udf('int')  # 리턴해주는 데이터 타입
def scale_score(score):
    return score * 20

In [53]:
# SQL문처리
spark.sql("SELECT restaurant_name, translate(restaurant_name) AS restaurant_name_en, \
            specialty, address, scale_score(score) AS score_scaled \
            FROM restaurants").show()

+---------------+--------------------+----------+------------------------------+------------+
|restaurant_name|  restaurant_name_en| specialty|                       address|score_scaled|
+---------------+--------------------+----------+------------------------------+------------+
|       진현가든|      jinhyeongadeun|삼치돌솥밥|      경기 오산시 양산로 340딩|        70.0|
|     대광생막창|daegwangsaengmagc...|  돼지막창|       경기 화성시 한신대길 99|        80.0|
|     찌개동아리|      jjigaedong-ali|  제육전골|   경기 오산시 한신대133번길 4|        70.0|
|       한판삼겹|      hanpansamgyeob|    항정살|   경기 오산시 양산로410번길 8|        70.0|
|           화락|              hwalag|      초밥|경기 오산시 양산로 347 대성...|        70.0|
|         해우리|              haeuli|  해물라면|  경기 오산시 한신대길 135 1층|        90.0|
|   행복한콩박사|haengboghankongbagsa|맑은순두부|경기 오산시 양산로398번길 8-11|        70.0|
+---------------+--------------------+----------+------------------------------+------------+

